In [1]:
# S-Box (fixed mapping based on the table)
S_BOX = {
    0x0: 0xE, 0x1: 0x4, 0x2: 0xD, 0x3: 0x1,
    0x4: 0x2, 0x5: 0xF, 0x6: 0xB, 0x7: 0x8,
    0x8: 0x3, 0x9: 0xA, 0xA: 0x6, 0xB: 0xC,
    0xC: 0x5, 0xD: 0x9, 0xE: 0x0, 0xF: 0x7
}

#P-box permutation (transposition of bits)
P_BOX = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

# Example Subkeys
SUBKEYS = [0x1F2A, 0x3C4D, 0x5E6F]

def s_box_substituion(nibble):
    """ Substitutes a 4-bit nibbke using the S-Box """
    return S_BOX[nibble]

def permutaion(block):
    """ Permutes a 16-bit block using the P-box """
    return sum(((block >> (i)) & 1) << (P_BOX[i] - 1) for i in range(16))

def key_mixing(block, subkey):
    """ Mixes a 16-bit block with a 16-bit subkey """
    return block ^ subkey

def round_function(block, subkey):
    """ Performs the operation of a single round: Substitution, Permutation, and Key Mixing """
    # Split block into 4 nibbles
    nibbles = [(block >> (i * 4)) & 0xF for i in range(4)]
    
    # Apply substitution to each nibble
    substituted = [s_box_substituion(nibble) for nibble in nibbles]
    
    # Concatenate the nibbles into a 16-bit block
    substituted_block = sum(substituted[i] << (i * 4) for i in range(4))
    
    # Permute the 16-bit block
    block = permutaion(substituted_block)
    
    # Mix the block with the subkey
    mix_block = key_mixing(block, subkey)
    
    return mix_block

def encrypt_block(plaintext):
    """ Encrypts a single block of plaintext using SPN cipher """    
    block = plaintext
    for subkey in SUBKEYS:
        block = round_function(block, subkey)
    return block

if __name__ == "__main__":
    plaintext = 0x1234
    ciphertext = encrypt_block(plaintext)
    print(f"Plaintext: {hex(plaintext)}")
    print(f"Ciphertext: {hex(ciphertext)}")





Plaintext: 0x1234
Ciphertext: 0x8128


In [2]:
# Inverse S-box (inverse mapping of provided S-box)
INVERSE_S_BOX = {v: k for k, v in S_BOX.items()}

def inv_s_box_substitution(nibble):
    """Substitutes a 4-bit nibble using the inverse S-Box"""
    return INVERSE_S_BOX[nibble]

def inverse_permutation(block):
    """Permutes a 16-bit block using the inverse P-box"""
    return sum(((block >> (P_BOX.index(i))) & 1) << (i - 1) for i in range(1, 17))

def inv_round_function(block, subkey):
    """Performs the inverse operation of a single round"""
    # First, undo the key mixing (XOR is its own inverse)
    block = block ^ subkey
    
    # Undo the permutation
    block = inverse_permutation(block)
    
    # Split block into 4 nibbles
    nibbles = [(block >> (i * 4)) & 0xF for i in range(4)]
    
    # Apply inverse substitution to each nibble
    inverted = [inv_s_box_substitution(nibble) for nibble in nibbles]
    
    # Concatenate the nibbles into a 16-bit block
    block = sum(inverted[i] << (i * 4) for i in range(4))
    
    return block

def decrypt(ciphertext):
    """Decrypts a single block of ciphertext using SPN cipher"""
    block = ciphertext
    
    # Process in reverse order of subkeys
    for subkey in reversed(SUBKEYS):
        block = inv_round_function(block, subkey)
    
    return block

if __name__ == "__main__":
    plaintext = 0x1234
    ciphertext = encrypt_block(plaintext)
    decrypted_text = decrypt(ciphertext)
    print(f"Plaintext: {hex(plaintext)}")
    print(f"Ciphertext: {hex(ciphertext)}")
    print(f"Decrypted Text: {hex(decrypted_text)}")

Plaintext: 0x1234
Ciphertext: 0x8128
Decrypted Text: 0x1234


In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# ===============================
# Neural Distinguisher Code
# ===============================

def int_to_bin_vector(x, bits=16):
    """Convert an integer to a binary vector (list of 0s and 1s) of length 'bits'."""
    return [(x >> i) & 1 for i in range(bits)]

def pair_to_vector(c, c_prime):
    """Concatenates two 16-bit integers into one 32-bit binary vector."""
    return int_to_bin_vector(c, 16) + int_to_bin_vector(c_prime, 16)

def build_network(input_dim=32):
    """Builds a simple feed-forward neural network for binary classification."""
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def TrainNetwork(n, epochs, delta_x):
    """
    Implements the training algorithm:
    
    1. TrainingInput = ∅, TrainingOutput = ∅.
    2. For i = 0, …, n:
         a. With probability > 0.5:
               - Choose m uniformly from {0,1}^16.
               - Compute c = Encrypt(m) and c′ = Encrypt(m ⊞ delta_x)
               - Add (c, c′) to TrainingInput and 1 to TrainingOutput.
         b. Else:
               - Choose c and c′ uniformly from {0,1}^16.
               - Add (c, c′) to TrainingInput and 0 to TrainingOutput.
    3. Train and return the neural network.
    """
    training_inputs = []
    training_outputs = []
    
    for i in range(n):
        if random.random() > 0.5:
            # Genuine pair (label 1)
            m = random.randint(0, 0xFFFF)
            c = encrypt_block(m)
            m_delta = (m + delta_x) & 0xFFFF  # addition modulo 2^16
            c_prime = encrypt_block(m_delta)
            training_inputs.append(pair_to_vector(c, c_prime))
            training_outputs.append(1)
        else:
            # Random pair (label 0)
            c = random.randint(0, 0xFFFF)
            c_prime = random.randint(0, 0xFFFF)
            training_inputs.append(pair_to_vector(c, c_prime))
            training_outputs.append(0)
    
    X = np.array(training_inputs, dtype=np.float32)
    y = np.array(training_outputs, dtype=np.float32)
    
    net = build_network(input_dim=32)
    net.fit(X, y, epochs=epochs, verbose=1)
    return net

def neural_distinguisher(n_queries, delta_x, tau, n_train, epochs):
    """
    Implements the neural distinguisher (Algorithm A3):
    
    1. h = 0.
    2. Train the network using TrainNetwork(n_train, epochs, delta_x).
    3. For i = 0, …, n_queries:
         a. Choose m uniformly from {0,1}^16.
         b. c = Oracle(m) = Encrypt(m) and c′ = Oracle(m ⊞ delta_x) = Encrypt(m ⊞ delta_x).
         c. If Net(c, c′) outputs 1 then h = h + 1.
    4. If h ≥ tau * n_queries, return 1 (real cipher detected); else return 0.
    """
    # Train the neural network distinguisher
    net = TrainNetwork(n_train, epochs, delta_x)
    
    h = 0
    for i in range(n_queries):
        m = random.randint(0, 0xFFFF)
        c = encrypt_block(m)
        m_delta = (m + delta_x) & 0xFFFF
        c_prime = encrypt_block(m_delta)
        input_vec = np.array([pair_to_vector(c, c_prime)], dtype=np.float32)
        # Network outputs a probability; we threshold at 0.5
        prediction = net.predict(input_vec, verbose=0)[0][0]
        if prediction >= 0.5:
            h += 1
    
    if h >= tau * n_queries:
        return 1
    else:
        return 0

# ===============================
# Main Execution
# ===============================
if __name__ == "__main__":
    # Test the cipher encryption/decryption
    print("=== SPN Cipher Test ===")
    plaintext = 0x1234
    ciphertext = encrypt_block(plaintext)
    decrypted_text = decrypt(ciphertext)
    print(f"Plaintext:     {hex(plaintext)}")
    print(f"Ciphertext:    {hex(ciphertext)}")
    print(f"Decrypted text:{hex(decrypted_text)}")
    
    # Parameters for the neural distinguisher
    n_train = 10000    # number of training examples
    epochs = 10        # number of training epochs
    delta_x = 0x0100   # the difference Δx (addition modulo 2^16)
    n_queries = 1000   # number of queries for the distinguisher
    tau = 0.6          # threshold: if at least tau * n_queries predictions are 1, output 1
    
    print("\n=== Neural Distinguisher ===")
    result = neural_distinguisher(n_queries, delta_x, tau, n_train, epochs)
    if result == 1:
        print("Output: 1 (Real cipher detected)")
    else:
        print("Output: 0 (Random function detected)")
